<a href="https://colab.research.google.com/github/mnijhuis-dnb/OpenSourceWorkshop-main/blob/master/Explainable_AI/analyse_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1EE5KO2VXEbxwZ2P0k-PP03NsNRL4PBPJ
!gdown 1atPsRYwb-mWq585wlz8iGtw5uaKck4N8

In [ ]:
!pip install eli5
!pip install shap
!pip install pdpbox

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/mortage_data.csv', index_col=0)

In [ ]:
float_columns = ['Monthly Reporting Period Original', 'Original Interest Rate', 'Original UPB', 
    'Origination Date', 'First Payment Date', 
    'Remaining Months To Maturity Original', 'Maturity Date', 
    'Original Loan to Value Ratio (LTV)', 'Original Combined Loan to Value Ratio (CLTV)', 'Number of Borrowers', 
    'Debt-To-Income (DTI)', 'Borrower Credit Score at Origination', 'Co-Borrower Credit Score at Origination', 
    'Zip Code Short', 'Mortgage Insurance Percentage', 
    'Loan Payment History Original', 'Total Principal Current Original', 'Total Writeoff', 'Borrower Age']

In [ ]:
one_hot_columns = ['Channel', 'Seller Name', 'Servicer Name', 'First Time Home Buyer Indicator', 'Loan Purpose', 
    'Property Type', 'Occupancy Status', 'Property State', 'Prepayment Penalty Indicator', 
    'Interest Only Loan Indicator Original', 'Current Loan Delinquency Status', 'Borrower Education Level', 
    'Borrower Employment', 'Relocation Mortgage Indicator', 'High Balance Loan Indicator', 
    'Borrower Assistance Plan Original', 
    'Alternative Delinquency Resolution Original']

In [ ]:
keep_columns = ['Monthly Reporting Period Original', 'Channel', 'Seller Name', 'Servicer Name', 'Original Interest Rate', 
    'Original UPB', 'Origination Date', 
    'First Payment Date', 'Remaining Months To Maturity Original', 
    'Maturity Date', 'Original Loan to Value Ratio (LTV)', 'Original Combined Loan to Value Ratio (CLTV)', 'Number of Borrowers', 
    'Debt-To-Income (DTI)', 'Borrower Credit Score at Origination', 'Co-Borrower Credit Score at Origination', 
    'First Time Home Buyer Indicator', 'Loan Purpose', 'Property Type', 'Occupancy Status', 'Property State', 
    'Zip Code Short', 'Mortgage Insurance Percentage', 
    'Prepayment Penalty Indicator', 'Interest Only Loan Indicator Original', 'Current Loan Delinquency Status', 
    'Loan Payment History Original', 'Total Principal Current Original', 'Relocation Mortgage Indicator', 
    'High Balance Loan Indicator', 'Borrower Assistance Plan Original', 
    'Alternative Delinquency Resolution Original', 'Total Writeoff', 'Borrower Age', 'Borrower Education Level', 
    'Borrower Employment']

In [ ]:
df.loc[df['Prepayment Penalty Indicator'].isna(),'Prepayment Penalty Indicator'] = 'nan'
df.loc[df['Interest Only Loan Indicator Original'].isna(),'Interest Only Loan Indicator Original'] = 'nan'
df.loc[df['Alternative Delinquency Resolution Original']==7.0,'Alternative Delinquency Resolution Original'] = '7'


In [ ]:
df_x = df[keep_columns].copy()
df_y = df['Total Writeoff'].fillna(0)

In [ ]:
for col in float_columns:
    df_x[col] = pd.to_numeric(df_x[col],errors='coerce',downcast='integer').fillna(-1)

In [ ]:
for col in one_hot_columns:
    df_x = pd.concat([df_x.drop(columns=[col]), pd.get_dummies(df_x[col], prefix=col + ' ')],axis=1)

In [ ]:
class DataProcessor(torch.utils.data.Dataset):
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X.astype(float))
            self.y = torch.from_numpy(y.astype(float))
      
    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [ ]:
class MLP(torch.nn.Module):

  def __init__(self):
    super().__init__()
    self.layers = torch.nn.Sequential(
      torch.nn.Linear(207, 100),
      torch.nn.ReLU(),
      torch.nn.Linear(100, 50),
      torch.nn.ReLU(),
      torch.nn.Linear(50, 10),
      torch.nn.ReLU(),
      torch.nn.Linear(10, 1)
    )

  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [ ]:
dataset = DataProcessor(df_x.values, df_y.values, scale_data=True)
model = torch.load('model.pkl')
model = model.float()

In [ ]:
model(dataset.X.float())